# Turicreate baseline
Более подробно [почитать](https://apple.github.io/turicreate/docs/userguide/recommender/) про библиотеку

In [1]:
import os
import pandas as pd
import numpy as np
import turicreate as tc

In [2]:
# precision at k
def precision_at_k(r_true_arr, k):
    return np.sum(r_true_arr[:k]) / k


# average precision at k
def average_precision_at_k(r_true_arr, k):
    apk = 0
    for n in range(0, k):
        apk += precision_at_k(r_true_arr, n + 1) * r_true_arr[n]
    if np.sum(r_true_arr[:k]) != 0:
        return (apk) / k
    else:
        return 0


# average normed precision at k
def average_normed_precision_at_k(r_true_arr, k, n_true):
    apk = 0
    apk_ideal = n_true / k
    
    for n in range(0, k):
        apk += precision_at_k(r_true_arr, n + 1) * r_true_arr[n]
    if np.sum(r_true_arr[:k]) != 0:
        return ((apk) / k) / apk_ideal
    else:
        return 0

In [3]:
df_clients = tc.SFrame.read_csv('../retailhero-uplift/data/clients.csv')
df_products = tc.SFrame.read_csv("../retailhero-uplift/data/products.csv")
df_purchases = tc.SFrame.read_csv("../retailhero-uplift/data/purchases.csv")

Finished parsing file /Users/arty.erokhin/projects/competition/retail_hero/retailhero-uplift/data/clients.csv

Parsing completed. Parsed 100 lines in 0.318353 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/arty.erokhin/projects/competition/retail_hero/retailhero-uplift/data/clients.csv

Parsing completed. Parsed 400162 lines in 0.367728 secs.

...

Read 37107078 lines. Lines per second: 762250

Read 40871601 lines. Lines per second: 754206

Read 45173555 lines. Lines per second: 758605

Finished parsing file /Users/arty.erokhin/projects/competition/retail_hero/retailhero-uplift/data/purchases.csv

Parsing completed. Parsed 45786568 lines in 60.1803 secs.

In [4]:
df_purchases[:2]

Columns:
	client_id	str
	transaction_id	str
	transaction_datetime	str
	regular_points_received	float
	express_points_received	float
	regular_points_spent	float
	express_points_spent	float
	purchase_sum	float
	store_id	str
	product_id	str
	product_quantity	float
	trn_sum_from_iss	float
	trn_sum_from_red	str

Rows: 2

Data:
+------------+----------------+----------------------+-------------------------+
| client_id  | transaction_id | transaction_datetime | regular_points_received |
+------------+----------------+----------------------+-------------------------+
| 000012768d |   7e3e2e3984   | 2018-12-01 07:12:45  |           10.0          |
| 000012768d |   7e3e2e3984   | 2018-12-01 07:12:45  |           10.0          |
+------------+----------------+----------------------+-------------------------+
+-------------------------+----------------------+----------------------+--------------+
| express_points_received | regular_points_spent | express_points_spent | purchase_sum |
+-------------------------+----------------------+----------------------+--------------+
|           0.0           |         0.0          |         0.0          |    1007.0    |
|           0.0           |         0.0          |         0.0          |    1007.0    |
+-------------------------+----------------------+----------------------+--------------+
+------------+------------+------------------+------------------+------------------+
|  store_id  | product_id | product_quantity | trn_sum_from_iss | trn_sum_from_red |
+------------+------------+------------------+------------------+------------------+
| 54a4a11a29 | 9a80204f78 |       2.0        |       80.0       |                  |
| 54a4a11a29 | da89ebd374 |       1.0        |       65.0       |                  |
+------------+------------+------------------+------------------+------------------+
[2 rows x 13 columns]

In [5]:
df_clients[:2]

client_id,first_issue_date,first_redeem_date,age,gender
000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F


In [6]:
df_products[:2]

Columns:
	product_id	str
	level_1	str
	level_2	str
	level_3	str
	level_4	str
	segment_id	float
	brand_id	str
	vendor_id	str
	netto	float
	is_own_trademark	int
	is_alcohol	int

Rows: 2

Data:
+------------+------------+------------+------------+------------+------------+
| product_id |  level_1   |  level_2   |  level_3   |  level_4   | segment_id |
+------------+------------+------------+------------+------------+------------+
| 0003020d3c | c3d3a8e8c6 | c2a3ea8d5e | b7cda0ec0c | 6376f2a852 |   123.0    |
| 0003870676 | e344ab2e71 | 52f13dac0c | d3cfe81323 | 6dc544533f |   105.0    |
+------------+------------+------------+------------+------------+------------+
+------------+------------+-------+------------------+------------+
|  brand_id  | vendor_id  | netto | is_own_trademark | is_alcohol |
+------------+------------+-------+------------------+------------+
| 394a54a7c1 | 9eaff48661 |  0.4  |        0         |     0      |
| acd3dd483f | 10486c3cf0 |  0.68 |        0         |     0      |
+------------+------------+-------+------------------+------------+
[2 rows x 11 columns]

In [7]:
# сделаем значения текстовыми, чтобы потом Turicreate сам сделал OHE
df_products["segment_id"] = df_products["segment_id"].astype(str)
df_products["is_alcohol"] = df_products["is_alcohol"].astype(str)
df_products["is_own_trademark"] = df_products["is_own_trademark"].astype(str)

In [8]:
# разбиваем на трейн и тест
training_data, validation_data = \
    tc.recommender.util.random_split_by_user(df_purchases, "client_id", "product_id", 
                                            max_num_users=10000, random_seed=42)

In [9]:
training_data.shape, validation_data.shape

((45556268, 13), (230300, 13))

In [10]:
columns = ['client_id',
'product_id']

item_columns = ['product_id',
 'level_1',
 'level_2',
 'level_3',
 'level_4',
 'segment_id',
 'brand_id',
 'vendor_id',
 'is_own_trademark',
 'is_alcohol']

In [11]:
# обучим модель с дополнительными данными о продуктах
model = tc.recommender.ranking_factorization_recommender.create(training_data[columns], 
                                                        user_id="client_id", 
                                                        item_id="product_id",
                                                        max_iterations=20,
                                                        item_data=df_products[item_columns])

Preparing data set.

Data has 45556268 observations with 400157 users and 43038 items.

Data prepared in: 15.5776s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training Predictive Error.

Final objective value: 0.574467

Final training Predictive Error: 0.263977

In [12]:
# рекомендации для пользователей из вадидационной выборки
recommended = model.recommend(validation_data["client_id"].unique(), k=30, 
                              exclude_known=False)

recommendations finished on 1000/9871 queries. users per second: 295.074

recommendations finished on 2000/9871 queries. users per second: 292.381

recommendations finished on 3000/9871 queries. users per second: 291.811

recommendations finished on 4000/9871 queries. users per second: 292.018

recommendations finished on 5000/9871 queries. users per second: 293.52

recommendations finished on 6000/9871 queries. users per second: 293.936

recommendations finished on 7000/9871 queries. users per second: 296.749

recommendations finished on 8000/9871 queries. users per second: 299.506

recommendations finished on 9000/9871 queries. users per second: 300.654

In [13]:
# датафрейм с покупками в реальности
reality = validation_data["client_id", "product_id"].to_dataframe()
reality.loc[:, "is_buyed"] = 1

# Metrics (precision@30, avg_precision@30, average_normed_precision@30)

In [14]:
rec_df = recommended.to_dataframe().merge(reality, on=["client_id", "product_id"], 
                                          how="left", sort=False).fillna(0)

In [15]:
# словарь с количеством покупок на валидации
real_dict = reality.groupby(by="client_id")["is_buyed"].sum().to_dict()

In [16]:
np.mean([precision_at_k(i, 30) for i in 
         rec_df.groupby(by="client_id", sort=False)["is_buyed"].apply(list)])

0.15021443285043734

In [17]:
np.mean([average_precision_at_k(i, 30) for i in 
         rec_df.groupby(by="client_id")["is_buyed"].apply(list)])

0.08843787115556434

In [18]:
np.mean([average_normed_precision_at_k(i, 30, real_dict.get(client, 0)) for client, i in 
         rec_df.groupby(by="client_id")["is_buyed"].apply(list).reset_index().values])

0.09879876704365677

# Сохраняем нашу модель

In [19]:
model.save("x5.model")

# Загрузка модели быстрая

In [20]:
%%time
model = tc.load_model("x5.model")

CPU times: user 770 ms, sys: 149 ms, total: 919 ms
Wall time: 480 ms


# Итого:
1. Мы обучили factorization_recommender из библиотеки turicreate;
2. Сделали валидационное и обучающее множества, получив на валидации результаты:
    - average_precision_at_k: 0.088
    - average_normed_precision_at_k: 0.099
3. Сохранили модель и использовали ее в решении;
4. Получили следующие результаты при загрузке нашего ответа:
    - check: 0.0907
    - public: 0.0754